# Heating Mesh Tally on CAD geometry made from Components

This constructs a reactor geometry from 3 Component objects each made from points.

The Component made include a breeder blanket, PF coil and a central column shield.

2D and 3D Meshes tally are then simulated to show nuclear heating, flux and tritium_production across the model.

This makes a 3D geometry and material for PF coil

In [1]:
import paramak


pf_coil = paramak. PoloidalFieldCoil(
    height=50,
    width=50,
    center_point=(725, 25),
    name='pf_coil'
)

This makes a 3D geometry and material for the centre column

In [2]:
center_column = paramak.CenterColumnShieldHyperbola(
    height=1200,
    inner_radius=50,
    mid_radius=100,
    outer_radius=150,
    name='center_column'
)

This makes a 3D geometry and material for breeder blanket. The azimuth_placement_angle argument is used to repeat the geometry around the Z axis at specified angles.

In [3]:
blanket = paramak.BlanketConstantThicknessArcH(
    inner_mid_point=(500, 0),
    inner_upper_point=(400, 300),
    inner_lower_point=(400, -300),
    thickness= 100,
    rotation_angle=40,
    azimuth_placement_angle=[0, 45, 90, 135, 180, 225, 270, 315],
    name='blanket'
)

This makes a reactor object from the three components

In [4]:
my_reactor = paramak.Reactor([blanket, pf_coil,center_column])

my_reactor.show()

Overwriting auto display for cadquery Workplane and Shape


At this stage we can export the reactor geometry as stp files and make them avaialbe from download and viewing in FreeCAD.

In [5]:
my_reactor.export_stp('my_reactor.stp')

from IPython.display import FileLink
display(FileLink('my_reactor.stp'))


*******************************************************************
******        Statistics on Transfer (Write)                 ******

*******************************************************************
******        Transfer Mode = 0  I.E.  As Is       ******
******        Transferring Shape, ShapeType = 0                      ******


/home/jshim/neutronics-workshop/tasks/task_12_CAD_mesh_fast_flux/my_reactor.stp

** WorkSession : Sending all data
 Step File Name : my_reactor.stp(3447 ents)  Write  Done


Combines and converts the stl file into a DAGMC h5m file that can be used for the model geometry

In [6]:
my_reactor.export_dagmc_h5m('dagmc.h5m')

import openmc_dagmc_wrapper as odw
geometry = odw.Geometry(h5m_filename='dagmc.h5m')

/home/jshim/brep_part_finder/brep_part_finder/core.py:75: UserWarning: No parts matching the specified volume +/- tolerances were found
  warnings.warn(
/home/jshim/brep_part_finder/brep_part_finder/core.py:114: UserWarning: No parts matching the specified bounding boxes were found
  warnings.warn("No parts matching the specified bounding boxes were found")


Info    : Meshing 1D...
Info    : [  0%] Meshing curve 1 (Circle)
Info    : [ 10%] Meshing curve 2 (Circle)
Info    : [ 10%] Meshing curve 3 (Line)
Info    : [ 10%] Meshing curve 4 (Circle)
Info    : [ 10%] Meshing curve 5 (Circle)
Info    : [ 10%] Meshing curve 6 (Line)
Info    : [ 10%] Meshing curve 7 (Circle)
Info    : [ 10%] Meshing curve 8 (Line)
Info    : [ 10%] Meshing curve 9 (Circle)
Info    : [ 10%] Meshing curve 10 (Circle)
Info    : [ 10%] Meshing curve 11 (Circle)
Info    : [ 20%] Meshing curve 12 (BSpline)
Info    : [ 20%] Meshing curve 13 (Circle)
Info    : [ 20%] Meshing curve 14 (Circle)
Info    : [ 20%] Meshing curve 15 (Circle)
Info    : [ 20%] Meshing curve 16 (Circle)
Info    : [ 20%] Meshing curve 17 (Circle)
Info    : [ 20%] Meshing curve 18 (Line)
Info    : [ 20%] Meshing curve 19 (Line)
Info    : [ 20%] Meshing curve 20 (Circle)
Info    : [ 20%] Meshing curve 21 (Circle)
Info    : [ 20%] Meshing curve 22 (Circle)
Info    : [ 30%] Meshing curve 23 (Line)
Info   

The next section uses materials from the neutronics_material_maker package https://github.com/fusion-energy/neutronics_material_maker/

This has an internal database that knows the isotopes and density of various materials.

The ```.openmc_material``` obtains an openmc.material object

In [7]:
# makes use of the previously created neutronics geometry (h5m file) and assigns
# actual materials to the material tags. 

import openmc_dagmc_wrapper as odw
import neutronics_material_maker as nmm

# this links the material tags in the dagmc h5m file with materials.
# these materials are input as strings so they will be looked up in the
# neutronics material maker package
material_tag_to_material_dict = {
    'mat_pf_coil': nmm.Material.from_library(name='copper'),
    'mat_blanket': nmm.Material.from_library(name='Li4SiO4'),
    'mat_center_column': nmm.Material.from_library(name='copper'),
}

geometry = odw.Geometry(h5m_filename='dagmc.h5m')

materials = odw.Materials(
    h5m_filename='dagmc.h5m',
    correspondence_dict=material_tag_to_material_dict
)

/home/jshim/openmc-1/openmc/mixin.py:67: IDWarning: Another Surface instance already exists with id=9999.
  warn(msg, IDWarning)
/home/jshim/openmc-1/openmc/mixin.py:67: IDWarning: Another Cell instance already exists with id=9999.
  warn(msg, IDWarning)


ValueError: material with tag mat_blanket was not found in the dagmc h5m file. The DAGMC file dagmc.h5m contains the following material tags ['mat_center_column', 'mat_pf_coil'].

This next step makes a simple point source. This uses native OpenMC commands but one could also use the openmc-plasma-source package https://github.com/fusion-energy/openmc-plasma-source/

In [ ]:
import openmc

# initialises a new source object
source = openmc.Source()

# sets the location of the source to x=0 y=0 z=0
source.space = openmc.stats.Point((100, 0, 0))

# sets the direction to isotropic
source.angle = openmc.stats.Isotropic()

# sets the energy distribution to 100% 14MeV neutrons
source.energy = openmc.stats.Discrete([14e6], [1])

This next section combines the geometry with the materials and specifies a 2D mesh tally that spans the entire geometry

In [ ]:
from dagmc_bounding_box import DagmcBoundingBox

# finds bounding box size from the geometry size
corners = DagmcBoundingBox('dagmc.h5m').corners()

tally1 = odw.MeshTally2D(
    mesh_resolution=(100, 100),
    bounding_box= corners,
    tally_type="(n,Xa)",
    plane='xz'
)

tallies = openmc.Tallies([tally1])

Sets the simulation intensity then combines the materials, geometry, settines and tallies into a neutronics model and runs the simulation.

In [ ]:
import openmc_plasma_source as ops

settings = odw.FusionSettings()
settings.batches = 4
settings.particles = 1000

# assigns a ring source of DT energy neutrons to the source using the
# openmc_plasma_source package, more details here
# https://github.com/fusion-energy/openmc-plasma-source/
settings.source = ops.FusionRingSource(fuel="DT", radius=350)


my_model = openmc.Model(
    materials=materials,
    geometry=geometry,
    settings=settings,
    tallies=tallies
)

!rm *.h5  # just removes old summary.h5 or statepoint files
statepoint_file = my_model.run()

The next section produces download links for:

- vtk files that contain the 3D mesh results (open with Paraview)
- png images that show the resuls of the 2D mesh tally

In [ ]:
from openmc_mesh_tally_to_vtk import write_mesh_tally_to_vtk
# importing a package for converting regular mesh tallies to vtk files
# more details here https://github.com/fusion-energy/openmc_mesh_tally_to_vtk


# assumes you have a statepoint file from the OpenMC simulation
statepoint = openmc.StatePoint('statepoint.4.h5')

# this shows the tallies present in the statepoint file
print(statepoint.tallies)

# loads up a tally from the statepoint using it's name
my_tally = statepoint.get_tally(name='(n,Xa)_on_2D_mesh_xz')

# converts the tally result into a VTK file
write_mesh_tally_to_vtk(
    tally=my_tally,
    filename = "(n,Xa)_on_2D_mesh_xz.vtk",
)

The code block below provides a download link for a VTK which can be opened with Paraview


In [ ]:
from IPython.display import FileLink
display(FileLink('n,Xa)_on_2D_mesh_xz.vtk'))